In [1]:
# Librairies
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import joblib


In [2]:
# Chargement des données
df = pd.read_csv("https://assets-datascientest.s3-eu-west-1.amazonaws.com/de/total/churn.csv")
df = df.reset_index()

In [3]:
# Identification des valeurs manquantes pour TotalCharges:
df = df.replace(' ', np.nan)
df['TotalCharges'] = df['TotalCharges'].astype('float64')

In [4]:
# On remplace les valeurs nulles par 0 pour la variable TotalCharges, ces données correspondent aux factures totales de nouveaux clients.
df.TotalCharges = df.TotalCharges.fillna(0)

In [5]:
# Mise à jour de la variable SeniorCitizen
df['SeniorCitizen'].replace({0:'No',1:'Yes'}, inplace=True)
# Mise à jour de la variable Churn
df['Churn'].replace({'No':0,'Yes':1}, inplace=True)
# Conversion de la variable TotalCharges en numérique
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

In [6]:
# Définition des variables pour la construction des pipelines
# Définition des variables catégorielles
colonnes_categorielles = ['gender',
                          'SeniorCitizen',
                          'Partner',
                          'Dependents',
                          'PhoneService',
                          'MultipleLines',
                          'OnlineSecurity',
                          'Contract',
                          'PaperlessBilling',
                          'PaymentMethod',
                          'StreamingMovies',
                          'StreamingTV',
                          'TechSupport',
                          'DeviceProtection',
                          'InternetService',
                          'OnlineBackup']

# Variables numériques
colonnes_numeriques =['tenure',
                      'MonthlyCharges',
                      'TotalCharges']

# Variable à supprimer
colonnes_a_supprimer=['customerID']

# Variable cible
colonne_cible = ['Churn']

In [7]:
# Suppression des colonnes qui non nécessaires aus modèles notemment 'customerID'
df = df.drop(colonnes_a_supprimer,axis='columns')

In [8]:
# Pipeline de construction du modèle et sauvegarde
# Jeu de test et d'entrainement
from sklearn.model_selection import train_test_split

# Varaibles explicatives
X = df.drop(colonne_cible, axis='columns')

# Variable cible
y = df[colonne_cible]
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, stratify=y, random_state = 0)

# Transformation des données et construction du pipeline du modèle de regressionlogistique
numeric_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='mean')),
       ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
       ('imputer', SimpleImputer(strategy='constant')),
       ('onehot',OneHotEncoder())
])

preprocessor = ColumnTransformer(
   transformers=[
    ('numeric', numeric_transformer, colonnes_numeriques),
    ('categorical', categorical_transformer, colonnes_categorielles)
]) 

pipeline_reglog = Pipeline(steps = [
               ('preprocessor', preprocessor),
               ('Oversampling', SMOTE()),
               ('regressor',LogisticRegression())
           ])
# Construction du modèle de regression logistique
model_reglog = pipeline_reglog.fit(X, y)
# Sauvegarde du modèle de regression logistique
joblib.dump(model_reglog, './model_regressionlogistique.pkl')

# Pipeline adboost
pipeline_adboost = Pipeline(steps = [
               ('preprocessor', preprocessor),
               ('Oversampling', SMOTE()),
               ('regressor',AdaBoostClassifier())
           ])
# Construction du modèle AdboostClassifier
model_adboost = pipeline_adboost.fit(X,y)
# Sauvegarde du modèle
joblib.dump(model_adboost, './model_AdboostClassifier.pkl')

C:\Users\c.nzie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\c.nzie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['./model_AdboostClassifier.pkl']